# Minimal, stripped-down version using Fast.ai
If you're new to Kaggle like me, you may have noticed the following frustrations:
1. No feedback loop; no ability to know what you've done wrong, or how to fix it, when something doesn't happen as expected.
2. Even worse, it looks like it's doing something for 30 minutes, and then you stumble into frustration #1.

So I took this [excellent kernel](https://www.kaggle.com/mentalwanderer/minimal-fast-ai-kit-for-image-classification) and stripped it down to only the bare necessities. Depending on your settings, it should take a half-hour to an hour to run.  From here, you and I can get fancier if we like, but at least we have run a successful submission on Kaggle.

NOTE THAT NOTHING IN HERE SHOULD BE CONSIDERED A BEST PRACTICE, OR EVEN NECESSARILY 'ACCEPTABLE PRACTICE'. In particular, I'm totally skipping EDA - which is a huge and fundamental part of any successful data science project, but I'm literally just focusing on the mechanics of getting a working Kaggle submission.

# Import packages

In [ ]:
from fastai import *
from fastai.vision import *
from torchvision.models import * 

import os
import pathlib
import matplotlib.pyplot as plt

# Data loading and preparation

In [ ]:
np.random.seed(123)

path = Path("../input")

tfms = get_transforms(do_flip = True,flip_vert = True)

In [ ]:
sz = 36
data = ImageDataBunch.from_csv(path, folder = 'train', csv_labels = "train_labels.csv",
                               test = 'test',suffix=".tif", size = sz, ds_tfms = tfms)
data.path = pathlib.Path('.')
stats = data.batch_stats()
data.normalize(stats)

# Model creation

In [ ]:
from sklearn.metrics import roc_auc_score

def auc_score(y_pred,y_true,tens=True):
    score = roc_auc_score(y_true,torch.sigmoid(y_pred)[:,1])
    if tens:
        score = tensor(score)
    return score

In [ ]:
learn = create_cnn(data,resnet18, pretrained = True,metrics = [auc_score ,accuracy])
learn.fit(epochs=3)

# Generating submission

As the order of the images loaded into `data` is not necessarily the same order as that in the required submission, we will need to rearrange the predictions on the test set.

The desired order for submission is that of `sample_submission.csv`. while the order of the test set loaded into `data` can be accessed by calling `learn.data.test_ds.items`. We will first create a dictionary assigning each image in the test to its prediction by the model, then call the keys in the order in  `sample_submission.csv`.

In [ ]:
submissions = pd.read_csv('../input/sample_submission.csv')
id_list = list(submissions.id)

In [ ]:
preds,y = learn.TTA(ds_type=DatasetType.Test)
pred_list = list(preds[:,1])

In [ ]:
print(learn.data.test_ds.items)

In [ ]:
pred_dict = dict((key, value.item()) for (key, value) in zip(learn.data.test_ds.items,pred_list))
pred_ordered = [pred_dict[Path('../input/test/' + id + '.tif')] for id in id_list]

In [ ]:
import time # Adding int(time.time() at end of file name so it's unique)
submissions = pd.DataFrame({'id':id_list,'label':pred_ordered})
submissions.to_csv("submission_weightdecay_{}.csv".format(int(time.time())),index = False)

# What now?

* Thanks again to [mentalwanderer](https://www.kaggle.com/mentalwanderer) for excellent kernel (who is not responsible for anything stupid I've done), and to the Fast.ai team for awesome and simple code.
* Check out mentalwanderer's original notebook for lots more detail and things that will provide a better end result: https://www.kaggle.com/mentalwanderer/minimal-fast-ai-kit-for-image-classification
* Check out the [Fast.ai library](https://docs.fast.ai/index.html) and [the v3 course](https://course.fast.ai/index.html).